In [1]:
import pandas as pd
import json
import os

# --- NUOVA FUNZIONE DI NORMALIZZAZIONE ---
def normalize_id(df, id_column='id'):
    """Converte in modo sicuro una colonna ID in una stringa pulita, rimuovendo i decimali."""
    # Forza numerico (gestisce errori), rimuove NaN, converte a int (via Int64 nullable) poi stringa
    df[id_column] = pd.to_numeric(df[id_column], errors='coerce')
    df = df.dropna(subset=[id_column])
    df[id_column] = df[id_column].astype('Int64')
    df[id_column] = df[id_column].astype(str)
    return df

# --- Configurazione ---
INPUT_DIR = "Test/case_law_csv"  # Assicurati che il percorso sia corretto
OUTPUT_DIR = "Test/neo4j_import_separated_tx" 

CASES_CSV = os.path.join(INPUT_DIR, "cases.csv")
CASE_DETAILS_CSV = os.path.join(INPUT_DIR, "case_texts.csv")
CITATIONS_CSV = os.path.join(INPUT_DIR, "citations.csv")

# === MODIFICA QUI: Puntiamo al file PULITO ===
LLM_DETAILS_JSON = "case_details_tx_CLEANED.json"
LLM_CONVICTION_JSON = "convictions_with_llm.json"

# File di output per Neo4j
FINAL_CASES_CSV = os.path.join(OUTPUT_DIR, "nodes_cases.csv")
FINAL_TEXT_CSV = os.path.join(OUTPUT_DIR, "nodes_text.csv")
FINAL_CITES_CSV = os.path.join(OUTPUT_DIR, "edges_citations.csv")
FINAL_HASTEXT_CSV = os.path.join(OUTPUT_DIR, "edges_has_text.csv")

os.makedirs(OUTPUT_DIR, exist_ok=True)

print("Inizio del processo di unione e separazione (Texas) - SAFE MODE...")

try:
    # --- 1. Carichiamo tutti i file ---
    print(f"Caricamento di {CASES_CSV}...")
    df_cases = pd.read_csv(CASES_CSV, dtype={'id': str})
    # Teniamo solo colonne utili
    df_cases = df_cases[['id', 'name', 'decision_date', 'court_name', 'jurisdiction_name_long', 'parties']]
    
    print(f"Caricamento di {CASE_DETAILS_CSV}...")
    df_details = pd.read_csv(CASE_DETAILS_CSV, dtype={'id': str})
    
    print(f"Caricamento di {LLM_DETAILS_JSON} (Dataset Pulito)...")
    with open(LLM_DETAILS_JSON, 'r', encoding='utf-8') as f:
        data_details_llm = json.load(f)
    df_details_llm = pd.DataFrame(data_details_llm)
    
    # Pulizia colonne extra se presenti
    if 'title' in df_details_llm.columns:
        df_details_llm = df_details_llm.drop(columns=['title'])

    print(f"Caricamento di {LLM_CONVICTION_JSON}...")
    with open(LLM_CONVICTION_JSON, 'r', encoding='utf-8') as f:
        data_conviction_llm = json.load(f)
    df_conviction_llm = pd.DataFrame(data_conviction_llm)

    # --- 1b. NORMALIZZIAMO TUTTI GLI ID ---
    print("Normalizzazione degli ID...")
    df_cases = normalize_id(df_cases, 'id')
    df_details = normalize_id(df_details, 'id')
    df_details_llm = normalize_id(df_details_llm, 'id')
    df_conviction_llm = normalize_id(df_conviction_llm, 'id')

    # --- 2. Uniamo (Merge) ---
    # STRATEGIA IMPORTANTE: Usiamo INNER JOIN sul dataset LLM PULITO.
    # Questo elimina automaticamente tutti i casi grezzi che non hanno superato la pulizia.
    
    print("Unione Master: Filtro basato sui casi puliti...")
    
    # 1. Unisci Metadati (Cases) con Dettagli LLM (Cleaned) -> INNER JOIN
    # Se un caso c'è nel CSV ma non nel JSON pulito, viene scartato qui.
    df_master = pd.merge(df_cases, df_details_llm, on='id', how='inner')
    
    # 2. Unisci il Testo Completo -> INNER JOIN
    df_master = pd.merge(df_master, df_details, on='id', how='inner')
    
    # 3. Unisci Convictions -> LEFT JOIN (Perché alcuni potrebbero mancare, ma non vogliamo perdere il caso)
    df_master = pd.merge(df_master, df_conviction_llm, on='id', how='left')

    print(f"Dimensione finale del DataFrame master (Casi Validi): {len(df_master)}")
    
    # Creiamo un SET di tutti gli ID validi per filtrare le citazioni
    valid_ids_set = set(df_master['id'])

    # --- 3. SEPARIAMO il DataFrame master in file di Nodi ---

    # File 1: Nodi :Case (leggeri)
    # Assicuriamoci che le colonne esistano (anche se vuote)
    case_columns = [
        'id', 'name', 'decision_date', 'court_name', 'jurisdiction_name_long',
        'parties', 'offense', 'punishment', 'decision', 'convicted'
    ]
    for col in case_columns:
        if col not in df_master.columns:
            df_master[col] = None
            
    df_nodes_cases = df_master[case_columns].copy()
    
    # Rinomina per Neo4j Headers
    df_nodes_cases = df_nodes_cases.rename(columns={
        'id': 'caseId:ID(Case)', 
        'decision': 'decisionSummary' # Rinomino per evitare confusione con keywords
    })
    
    # Gestione booleani per Neo4j (True/False invece di 1/0 o float)
    df_nodes_cases['convicted'] = df_nodes_cases['convicted'].astype(str).replace({'nan': '', 'None': ''})
    
    df_nodes_cases.to_csv(FINAL_CASES_CSV, index=False)
    print(f"\n--- SALVATO: File Nodi :Case (leggeri) --- {len(df_nodes_cases)} righe.")

    # File 2: Nodi :CaseText (pesanti)
    text_columns = ['id', 'head_matter', 'opinions']
    # Se head_matter non c'è, creala vuota
    for col in text_columns:
        if col not in df_master.columns:
            df_master[col] = ""

    df_nodes_text = df_master[text_columns].copy()
    df_nodes_text = df_nodes_text.rename(columns={
        'id': 'textId:ID(CaseText)', 
        'opinions': 'text'
    })
    df_nodes_text.to_csv(FINAL_TEXT_CSV, index=False)
    print(f"\n--- SALVATO: File Nodi :CaseText (pesanti) --- {len(df_nodes_text)} righe.")
    
    # --- 4. Creiamo i file delle RELAZIONI ---

    # File 3: Relazioni :CITES
    print(f"\nCaricamento di {CITATIONS_CSV} per le relazioni :CITES...")
    
    # Controllo nomi colonne citazioni (adattalo se il tuo CSV ha nomi diversi come 'citing_case_id')
    try:
        df_edges_cites = pd.read_csv(CITATIONS_CSV, dtype=str)
        # Standardizziamo i nomi se necessario
        if 'citing_case_id' in df_edges_cites.columns:
            df_edges_cites = df_edges_cites.rename(columns={'citing_case_id': 'id1', 'cited_case_id': 'id2'})
    except Exception as e:
        print(f"Errore lettura citazioni: {e}")
        df_edges_cites = pd.DataFrame(columns=['id1', 'id2'])

    df_edges_cites = normalize_id(df_edges_cites, 'id1')
    df_edges_cites = normalize_id(df_edges_cites, 'id2')

    # Filtro "matched" se presente
    if 'matched' in df_edges_cites.columns:
        df_edges_cites = df_edges_cites[df_edges_cites['matched'].astype(str).str.lower() == 'true'].copy()
    
    # --- FILTRO FONDAMENTALE ---
    # Manteniamo SOLO le relazioni dove SIA id1 CHE id2 esistono nel nostro set MASTER.
    initial_edges = len(df_edges_cites)
    df_edges_cites = df_edges_cites[
        df_edges_cites['id1'].isin(valid_ids_set) & 
        df_edges_cites['id2'].isin(valid_ids_set)
    ]
    final_edges = len(df_edges_cites)
    
    print(f"Citazioni filtrate: da {initial_edges} a {final_edges} (Rimossi {initial_edges - final_edges} link a casi sporchi/inesistenti).")

    df_edges_cites = df_edges_cites.rename(columns={
        'id1': ':START_ID(Case)',
        'id2': ':END_ID(Case)'
    })
    df_edges_cites[':TYPE'] = 'CITES'
    df_edges_cites = df_edges_cites[[':START_ID(Case)', ':END_ID(Case)', ':TYPE']]
    
    df_edges_cites.to_csv(FINAL_CITES_CSV, index=False)
    print(f"\n--- SALVATO: File Relazioni :CITES --- {len(df_edges_cites)} righe.")

    # File 4: Nuove Relazioni :HAS_TEXT
    df_edges_hastext = df_master[['id']].copy()
    df_edges_hastext = df_edges_hastext.rename(columns={
        'id': ':START_ID(Case)'
    })
    df_edges_hastext[':END_ID(CaseText)'] = df_edges_hastext[':START_ID(Case)']
    df_edges_hastext[':TYPE'] = 'HAS_TEXT'
    
    df_edges_hastext.to_csv(FINAL_HASTEXT_CSV, index=False)
    print(f"\n--- SALVATO: File Relazioni :HAS_TEXT --- {len(df_edges_hastext)} righe.")

    print("\n SUCCESS: Processo completato. I file pronti per Neo4j sono in:")
    print(f"   -> {OUTPUT_DIR}")

except FileNotFoundError as e:
    print(f"\n ERRORE: File non trovato: {e.filename}")
except Exception as e:
    print(f"\n ERRORE GENERICO: {e}")

Inizio del processo di unione e separazione (Texas) - SAFE MODE...
Caricamento di Test/case_law_csv\cases.csv...
Caricamento di Test/case_law_csv\case_texts.csv...
Caricamento di case_details_tx_CLEANED.json (Dataset Pulito)...
Caricamento di convictions_with_llm.json...
Normalizzazione degli ID...
Unione Master: Filtro basato sui casi puliti...
Dimensione finale del DataFrame master (Casi Validi): 7791

--- SALVATO: File Nodi :Case (leggeri) --- 7791 righe.

--- SALVATO: File Nodi :CaseText (pesanti) --- 7791 righe.

Caricamento di Test/case_law_csv\citations.csv per le relazioni :CITES...
Citazioni filtrate: da 74756 a 5405 (Rimossi 69351 link a casi sporchi/inesistenti).

--- SALVATO: File Relazioni :CITES --- 5405 righe.

--- SALVATO: File Relazioni :HAS_TEXT --- 7791 righe.

 SUCCESS: Processo completato. I file pronti per Neo4j sono in:
   -> Test/neo4j_import_separated_tx


C:\Users\claud\AppData\Local\Temp\ipykernel_29672\4182158790.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[id_column] = df[id_column].astype('Int64')
C:\Users\claud\AppData\Local\Temp\ipykernel_29672\4182158790.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[id_column] = df[id_column].astype(str)
